In [7]:
import time  # Nhập khẩu thư viện time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import time
confusion_matrix

<function sklearn.metrics._classification.confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)>

In [8]:
df = pd.read_excel('employ_clean.xlsx')

In [9]:

df['StartDate'] = pd.to_datetime(df['StartDate'], errors='coerce')
df['ExitDate'] = pd.to_datetime(df['ExitDate'], errors='coerce')

# Tính số năm làm việc (Work Duration)
df['WorkDuration'] = (df['ExitDate'].fillna(pd.to_datetime('today')) - df['StartDate']).dt.days / 365.25

label_encoder = LabelEncoder()
df['IsEmployeeLeft'] = label_encoder.fit_transform(df['IsEmployeeLeft'])
df['GenderCode'] = label_encoder.fit_transform(df['GenderCode'])

df = pd.get_dummies(df, columns=['Performance Score','JobFunctionDescription'], drop_first=True)

X = df[['Age', 'GenderCode', 'WorkDuration', 'Current Employee Rating'] + [col for col in df.columns if 'Performance Score' in col or 'JobFunctionDescription' in col]]

y = df['IsEmployeeLeft']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

start_time_rf = time.time()

# Xây dựng mô hình Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Dự đoán và đánh giá mô hình
y_pred_rf = rf_model.predict(X_test)

# Tính toán độ chính xác
accuracy_rf = accuracy_score(y_test, y_pred_rf)

# Tính thời gian huấn luyện
training_time_rf = time.time() - start_time_rf


C:\Users\luan0\AppData\Local\Temp\ipykernel_3840\3272357571.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ExitDate'] = pd.to_datetime(df['ExitDate'], errors='coerce')


In [10]:
conf_matrix = confusion_matrix(y_test, y_pred_rf)
print(conf_matrix)
# Giải thích các giá trị trong ma trận nhầm lẫn
TN, FP, FN, TP = conf_matrix.ravel()

[[376  46]
 [127 351]]


TP số rời đi là 1 và họ rời đi 354
TN dự đoán cho ở lại và họ đã ở lại 371
FN dụ đoán ở lại nhưng rời đi 124
FP dự đoán rời đi nhưng họ ơ lại 51

In [11]:

training_time_rf = time.time() - start_time_rf
# Hiển thị kết quả
print(f'Training Time (Random Forest): {training_time_rf:.2f} seconds')
classification_rep_dt = classification_report(y_test, y_pred_rf)
print(classification_rep_dt)
# Kết quả chi tiết về mô hình (precision, recall, f1-score, support)

Training Time (Random Forest): 0.32 seconds
              precision    recall  f1-score   support

           0       0.75      0.89      0.81       422
           1       0.88      0.73      0.80       478

    accuracy                           0.81       900
   macro avg       0.82      0.81      0.81       900
weighted avg       0.82      0.81      0.81       900



In [12]:
from sklearn.metrics import cohen_kappa_score
# Tính Cohen's Kappa
kappa_value = cohen_kappa_score(y_test, y_pred_rf)
print(f'Cohen\'s Kappa: {kappa_value:.4f}')

Cohen's Kappa: 0.6184
